Dataset source: https://www.kaggle.com/datasets/thedevastator/better-recipes-for-a-better-life?select=recipes.csv

In [96]:
import pandas as pd
import numpy as np
import re
import random

In [2]:
data = pd.read_csv("recipes.csv")
data.fillna("UNK", inplace=True)
data.head()

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,UNK,UNK,UNK,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,UNK,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


In [89]:
cuisine_data = pd.read_csv('../data/cooking_data(cuisine).csv', sep=';')
cuisine_data

,name,image_id
0,Greek,1
1,French,2
2,Indian,3
3,Italian,4
4,Japanese,5
5,Chinese,6
6,Mexican,7
7,German,8
8,Korean,9
9,Turkish,10


In [90]:
cuisines = []
for row in cuisine_data.itertuples(index=True, name='Pandas'):
    cuisines.append(getattr(row, 'name'))

print(cuisines)

['Greek', 'French', 'Indian', 'Italian', 'Japanese', 'Chinese', 'Mexican', 'German', 'Korean', 'Turkish']


In [82]:
def clean_text(text):
    text = text.replace(u"'", u'')
    return text

def get_time(text):
    if text == "UNK":
        return 30 # default time
    if "hrs" in text:
        temp_h = text.split("hrs")
        hours = int(temp_h[0])
        if "mins" in text:
            temp_m = temp_h[1].split("mins")
            mins = int(temp_m[0])
            return hours*60 + mins
        else:
            return hours*60
    else:
        temp = text.split("mins")
        return int(temp[0])

def get_instr_list(text):
    steps = text.split("\n\n")[0].split(".")
    instr = []
    for step in steps:
        if step != '':
            instr.append(step.strip())
    return instr

def get_food_team_long_name(text):
    return text.split("/")[-2].strip()

# bad function needs edit 
def get_ingredients(text):
    text = text.replace('(', '<')
    text = text.replace(')', '>')
    temp = re.sub('<[^>]+>', '', text)
    l = temp.split(",")
    edited = []
    
    for seq in l:
        t = seq.split(" ")
        t = list(filter(None, t))
        if len(t) >= 3:
            line = ""
            for i in range(2, len(t)):
                line = line + " " + t[i]
            edited.append(line.strip())
            
    return edited

def get_food_info(text):
    dict = {}
    search = ["Total Fat", "Protein", "Total Carbohydrate"]
    show = ["fat", "protein", "carbohydrate"]
    for item in show:
        dict[item] = 0
    for i in range(0, len(search)):
        if search[i] in text:
            temp = int(text.split(search[i])[1].split("g")[0].strip())
            dict[show[i]] = temp
    return dict

In [119]:
# create dictionaries to become dataframes
image_dict = {
    "id": [],
    "description": [],
    "image_url": []
}

food_team_dict = {
    "short_name":[],
    "description":[],
    "long_name":[],
    "image_id":[]
}

ingredient_dict = {
    "name":[],
    "cal_per_100":[],
    "food_team_short_name":[],
    "image_id":[]
}

recipe_dict = {
    "type":[],
    "difficulty":[],
    "name":[],
    "description":[],
    "prep_time_m":[],
    "cook_time_m":[],
    "cuisine_name":[],
    "ingredient_name":[],
    "portions":[],
    "image_id":[]
}

food_info_dict = {
    "fat":[],
    "protein":[],
    "carbohydrate":[],
    "total_calories":[],
    "recipe_name":[],
    "recipe_cuisine_name":[]
}

step_dict = {
    "number":[],
    "description":[],
    "recipe_name":[],
    "recipe_cuisine_name":[]
}


In [120]:
types = ['Cooking', 'Pastry']
cuisine_selection = []
for i in range(10):
    for j in range(2):
        cuisine_selection.append(cuisines[i])
print(cuisine_selection)

['Greek', 'Greek', 'French', 'French', 'Indian', 'Indian', 'Italian', 'Italian', 'Japanese', 'Japanese', 'Chinese', 'Chinese', 'Mexican', 'Mexican', 'German', 'German', 'Korean', 'Korean', 'Turkish', 'Turkish']


In [121]:
RECIPES = 20
base_id = 1000

counter = 0
for row in data.itertuples(index=True, name='Pandas'):
    if counter < RECIPES:
        recipe_name = clean_text(getattr(row, 'recipe_name'))
        prep_time = clean_text(getattr(row, 'prep_time'))
        cook_time = clean_text(getattr(row, 'cook_time'))
        portions = getattr(row, 'servings')
        directions = clean_text(getattr(row, 'directions'))
        img_src = getattr(row, 'img_src')
        cuisine_path = clean_text(getattr(row, 'cuisine_path'))
        ingredients = clean_text(getattr(row, 'ingredients'))
        nutrition = clean_text(getattr(row, 'nutrition'))

        # images for recipes
        image_dict["id"].append(base_id)
        image_dict["description"].append("-")
        image_dict["image_url"].append(img_src)

        food_team_dict["short_name"].append(get_food_team_long_name(cuisine_path))
        food_team_dict["description"].append("-")
        food_team_dict["long_name"].append(get_food_team_long_name(cuisine_path))
        food_team_dict["image_id"].append(1)

        for ing in get_ingredients(ingredients):
            if not(ing in ingredient_dict["name"]):
                ingredient_dict["name"].append(ing)
                ingredient_dict["cal_per_100"].append(20)
                ingredient_dict["food_team_short_name"].append(get_food_team_long_name(cuisine_path))
                ingredient_dict["image_id"].append(1)

        recipe_dict["type"].append(random.choice(types))
        recipe_dict["difficulty"].append(random.randint(1, 5))
        recipe_dict["name"].append(recipe_name)
        recipe_dict["description"].append("-")
        recipe_dict["prep_time_m"].append(get_time(prep_time))
        recipe_dict["cook_time_m"].append(get_time(cook_time))
        recipe_dict["cuisine_name"].append(cuisine_selection[counter])
        recipe_dict["ingredient_name"].append("-")
        recipe_dict["portions"].append(portions)
        recipe_dict["image_id"].append(base_id)

        fi = get_food_info(nutrition)
        food_info_dict["fat"].append(fi["fat"])
        food_info_dict["protein"].append(fi["protein"])
        food_info_dict["carbohydrate"].append(fi["carbohydrate"])
        food_info_dict["total_calories"].append(0)
        food_info_dict["recipe_name"].append(recipe_name)
        food_info_dict["recipe_cuisine_name"].append(cuisine_selection[counter])
        
        number = 1
        for step in get_instr_list(directions):
            step_dict["number"].append(number)
            step_dict["description"].append(step)
            step_dict["recipe_name"].append(recipe_name)
            step_dict["recipe_cuisine_name"].append(cuisine_selection[counter])
            number+=1
        
        counter+=1
        base_id+=1
    else:
        break


In [122]:
image_df = pd.DataFrame(image_dict)
image_df.to_csv("image.csv", index=False)

food_team_df = pd.DataFrame(food_team_dict)
food_team_df.to_csv("food_team.csv", index=False)

ingredient_df = pd.DataFrame(ingredient_dict)
ingredient_df.to_csv("ingredient.csv", index=False)

recipe_df = pd.DataFrame(recipe_dict)
recipe_df.to_csv("recipe.csv", index=False)

food_info_df = pd.DataFrame(food_info_dict)
food_info_df.to_csv("food_info.csv", index=False)

step_df = pd.DataFrame(step_dict)
step_df.to_csv("step.csv", index=False, sep=';')